In [1]:
import json
import os
from openai import OpenAI

In [2]:
from google.colab import userdata
Nvidia_API_Key=userdata.get('Nvidia_API_Key')
os.environ["Nvidia_API_Key"] = Nvidia_API_Key

In [3]:
client = OpenAI(
    base_url="https://integrate.api.nvidia.com/v1",
    api_key=os.environ['Nvidia_API_Key']  # Accessing the secret key
)

In [4]:
input_file_path = '/content/modified_refined_pfd_descriptions.json'
output_file_path = '/content/scored_modified_refined_pfd_descriptions.json'

In [5]:
error_log = []  # List to track errors

# Open the input JSON file and load the data
try:
    with open(input_file_path, 'r') as file:
        data = json.load(file)

    results = []  # To store the processed results

    # Iterate over each chemical entry in the JSON
    for entry in data:
        chemical_name = entry.get('chemical_name', 'Unknown Chemical')
        question = entry.get('question', '')  # Getting the question
        response = entry.get('response', '')  # Getting the response

        if not response:
            print(f"Skipping {chemical_name} due to missing 'response'.")
            continue  # Skip to the next entry if response is missing

        # Prepare the messages for the Nemotron model
        messages = [
            {"role": "user", "content": question},
            {"role": "assistant", "content": response}
        ]

        try:
            # Call the Nemotron model to get scores
            completion = client.chat.completions.create(
                model="nvidia/nemotron-4-340b-reward",
                messages=messages
            )

            # Access the scores from the API response
            scores_message = completion.choices[0].message[0].content
            scores = scores_message.strip()  # Clean up any whitespace

            # Prepare a dictionary with the result for this chemical
            scores_list = {
                "chemical_name": chemical_name,
                "question": question,
                "response": response,
                "scores": scores
            }

            # Append the result to the list of results
            results.append(scores_list)

            print(f"Successfully processed and scored: {chemical_name}")

        except Exception as e:
            # Log any errors that occur during the model call
            error_log.append({"chemical_name": chemical_name, "error": str(e)})
            print(f"Error processing {chemical_name}: {e}")

    # Save the results to the output JSON file
    with open(output_file_path, 'w') as output_file:
        json.dump(results, output_file, indent=4)

    print(f"Processing complete. Results saved to {output_file_path}")

except Exception as e:
    # Log any errors that occur during reading the input file
    error_log.append({"file": input_file_path, "error": str(e)})
    print(f"Error reading input JSON file: {e}")

# If there were any errors, save the error log to a file
if error_log:
    with open("error_log.json", 'w') as error_file:
        json.dump(error_log, error_file, indent=4)
    print("Some errors occurred. Check 'error_log.json' for details.")

Successfully processed and scored: Propylene glycol propyl ether
Successfully processed and scored: Di(propylene glycol) butyl ether
Successfully processed and scored: Di(propylene glycol) dimethyl ether
Successfully processed and scored: Propylene glycol monomethyl ether acetate
Successfully processed and scored: Di(propylene glycol) propyl ether
Successfully processed and scored: Ethylene glycol monophenyl ether
Successfully processed and scored: Propylene glycol diacetate
Successfully processed and scored: Di(propylene glycol) methyl ether acetate
Successfully processed and scored: Diethylene glycol butyl ether
Successfully processed and scored: [3-(2-Aminoethylamino)propyl]trimethoxysilane
Successfully processed and scored: 2,2,4,4,6,6,8,8,10,10,12,12-Dodecamethylcyclohexasiloxane
Successfully processed and scored: 1-Dodecylpyridinium chloride hydrate
Successfully processed and scored: Dichloromethylsilane
Successfully processed and scored: Vespel
Successfully processed and scored: